In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [2]:
pip install missingpy

Note: you may need to restart the kernel to use updated packages.


Importing Data

In [3]:
import missingpy

data=pd.read_excel("creditfulltrainingdatamodified.xlsx")

data.replace(['m'],np.NaN,inplace=True)

data.head()

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,mvar47b,mvar48,default_ind
0,230032,1696.0,1.6541,0.000,0.0,0.0,0.0,6015.0,322.0,40369.0,...,82.547,0.08696,10.0,0.63899,NaN,0.0,1,0,10,0
1,230033,1846.0,0.8095,0.000,0.0,0.0,102.0,7532.0,3171.0,18234.0,...,NaN,0.00000,13.0,0.63836,NaN,NaN,0,1,732,1
2,230034,1745.0,0.4001,0.000,0.0,0.0,NaN,2536.0,NaN,NaN,...,29.290,0.00000,1.0,1.00000,NaN,0.0,1,0,89,1
3,230035,1739.0,0.2193,0.000,0.0,0.0,1982.0,26440.0,4955.0,20316.0,...,NaN,0.15385,3.0,0.53241,0.0,0.0,0,1,3,0
4,230036,1787.0,0.0118,0.225,0.0,0.0,5451.0,5494.0,5494.0,7987.0,...,NaN,0.00000,1.0,0.92665,NaN,NaN,0,1,5,0


In [4]:
data = data.drop('application_key', axis = 1)

data.head()

,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,...,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,mvar47b,mvar48,default_ind
0,1696.0,1.6541,0.000,0.0,0.0,0.0,6015.0,322.0,40369.0,18414.0,...,82.547,0.08696,10.0,0.63899,NaN,0.0,1,0,10,0
1,1846.0,0.8095,0.000,0.0,0.0,102.0,7532.0,3171.0,18234.0,13664.0,...,NaN,0.00000,13.0,0.63836,NaN,NaN,0,1,732,1
2,1745.0,0.4001,0.000,0.0,0.0,NaN,2536.0,NaN,NaN,2536.0,...,29.290,0.00000,1.0,1.00000,NaN,0.0,1,0,89,1
3,1739.0,0.2193,0.000,0.0,0.0,1982.0,26440.0,4955.0,20316.0,37013.0,...,NaN,0.15385,3.0,0.53241,0.0,0.0,0,1,3,0
4,1787.0,0.0118,0.225,0.0,0.0,5451.0,5494.0,5494.0,7987.0,4696.0,...,NaN,0.00000,1.0,0.92665,NaN,NaN,0,1,5,0


Imputing using missForest

In [5]:
from missingpy import MissForest
impute = MissForest()
data = impute.fit_transform(data)

Iteration: 0
Iteration: 1
Iteration: 2


In [13]:
X = data[:,:49]
Y = data[:,49]

In [15]:
Y.shape

(83000,)

In [16]:
from sklearn import preprocessing
X = preprocessing.scale(X)

Setting up XGboost model

In [1]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,Y,test_size=0.3,random_state=0)

import xgboost as xgb
model = xgb.XGBClassifier(max_depth=7,
                           min_child_weight=1,
                           learning_rate=0.01,
                           n_estimators=700,
                           silent=True,
                           objective='binary:logistic',
                           gamma=3,
                           max_delta_step=10000,
                           subsample=1,
                           colsample_bytree=1,
                           colsample_bylevel=1,
                           reg_alpha=50,
                           reg_lambda=10,
                           scale_pos_weight=3.5,
                           seed=1,
                           missing=None)

NameError: name 'X' is not defined

In [18]:
model=model.fit(X_train, y_train) #Training the model

In [19]:
y_pred = model.predict(X_test) #Prediciting output on test data

In [20]:
#Metrics:

#Accuracy
#F1 score
#Balanced Accuracy
#Precision Score
#Recall Score

y_pred=model.predict(X_test)
print(sklearn.metrics.accuracy_score(y_test, y_pred))
print(sklearn.metrics.f1_score(y_test, y_pred))
print(sklearn.metrics.balanced_accuracy_score(y_test, y_pred))
print(sklearn.metrics.precision_score(y_test, y_pred))
print(sklearn.metrics.recall_score(y_test, y_pred))

0.7042168674698795
0.5902186613253213
0.7136838272219596
0.49257057949479943
0.7361554476058293


In [21]:
#Confusion Matrix

from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[12231,  5464],
       [ 1901,  5304]], dtype=int64)